In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [2]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [3]:
symbols = ["MES", "6B"]
df = pth.SierraChartData.pull(
    symbols,
    timeframe="1min",
    start="2024-11-01",
    end="2024-12-01",
)
df

c:\Users\spask\miniconda3\Lib\site-packages\vectorbtpro\data\base.py:1728: VBTWarning: Symbols have mismatching index. Setting missing data points to NaN.


In [4]:
close = df.data['MES'].close
close

timestamp
2024-11-01 00:00:00+00:00    5808.50
2024-11-01 00:01:00+00:00    5808.75
2024-11-01 00:02:00+00:00    5808.50
2024-11-01 00:03:00+00:00    5808.50
2024-11-01 00:04:00+00:00    5808.75
                              ...   
2024-12-01 23:55:00+00:00    6115.00
2024-12-01 23:56:00+00:00    6115.25
2024-12-01 23:57:00+00:00    6115.75
2024-12-01 23:58:00+00:00    6116.00
2024-12-01 23:59:00+00:00    6115.75
Name: close, Length: 28637, dtype: float64

### Indicator

In [67]:
def simple_indicator(close, period=10, period_band=60, multiplier=1.):
    def count_up_down_ratio(s):
        up = (s.diff() > 0).sum()
        down = (s.diff() < 0).sum()
        return (up - down) / (up + down) if (up + down) != 0 else np.nan

    indicator = pd.Series(close).rolling(window=period).apply(count_up_down_ratio)
    band_window = indicator.rolling(window=period_band)
    m = band_window.mean()
    std = band_window.std()
    upper = m + multiplier * std
    lower = m - multiplier * std
    return indicator, upper, lower


@njit
def simple_indicator_nb(close, period=10, period_band=60, multiplier=1.):  #1000x faster :X
    indicator = np.full(close.shape[0], np.nan)
    upper, lower = np.full(close.shape[0], np.nan), np.full(close.shape[0], np.nan)

    for i in range(period, close.shape[0]):
        window = close[i-period: i]
        diff = window[1:] - window[:-1]
        up, down = np.sum(diff >= 0), np.sum(diff < 0)
        indicator[i] = (up - down) / (up + down) if (up + down != 0) else np.nan
    
    for i in range(period_band + period, indicator.shape[0]):
        window = indicator[i - period_band: i]
        m = window.mean()
        std = window.std()
        upper[i] = m  + multiplier * std
        lower[i] = m - multiplier * std

    return indicator, upper, lower

In [129]:
def make_indicator_with_func(indicator_func):
    SimpleIndicator = vbt.IndicatorFactory(
        class_name="SimpleIndicator",
        short_name="si",
        input_names=["close"],
        param_names=["period", "period_band", "multiplier"],
        output_names=["indicator", "upper", "lower"],
    ).with_apply_func(indicator_func, takes_1d=True, period=10, period_band=60, multiplier=1.)
    return SimpleIndicator

In [62]:
%%timeit
make_indicator_with_func(simple_indicator).run(close)

4.24 s ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [133]:
%%timeit
make_indicator_with_func(simple_indicator_nb).run(close, period=10)

11.8 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [139]:
indicator_nb =  make_indicator_with_func(simple_indicator_nb).run(close)

show_last = 2000
fig = indicator_nb.indicator[-show_last:].vbt.plot()
indicator_nb.upper[-show_last:].vbt.plot(fig=fig)
indicator_nb.lower[-show_last:].vbt.plot(fig=fig)

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e831233a-fcdc-4c62-91d5-fb3fcc1f06fc',
              'x': array([datetime.datetime(2024, 11, 28, 7, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 28, 7, 59, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 28, 8, 0, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([0.55555556, 0.55555556, 0.77777778, ..., 0.11111111, 0.33333333,
                          0.33333333])},
             {'name': 'close',
              's

### Pipeline

In [280]:
def pipeline(close, period=10, period_band=60, multiplier=1.):
    SimpleIndicator =  make_indicator_with_func(simple_indicator_nb)
    indicator = SimpleIndicator.run(
        close,
        period=period,
        period_band=period_band,
        multiplier=multiplier,
        param_product=True,
    )
    long_entries = indicator.indicator < indicator.upper
    long_exits = indicator.indicator > indicator.lower
    short_entries = long_exits
    short_exits = long_entries
    pf = vbt.Portfolio.from_signals(
        close,
        long_entries=long_entries,
        long_exits=long_exits,
        short_entries=short_entries,
        short_exits=short_exits,
        fees=0.0,
        # freq="1d",
    )
    return pf

In [257]:
%%timeit
pipeline(df.data['MES'].close)

27.3 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [281]:
pf = pipeline(df.data['MES'].close, period=np.arange(10, 100, 10), period_band=np.arange(50, 200, 10), multiplier=np.arange(1,3,.5))
pf

 39%|###9      | 213/540 [00:02<00:03, 106.33it/s, si_period=40, si_period_band=130, si_multiplier=1.5]

In [ ]:
pf.value.iloc[-1].vbt.volume()

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hovertemplate': ('si_period: %{x}<br>si_period_b' ... 'value: %{value}<extra></extra>'),
              'opacity': 0.2,
              'surface': {'count': 15},
              'type': 'volume',
              'uid': 'a8011c7b-90b8-4b14-9e58-fb3f46c87081',
              'value': array([ 95.70432477,  98.06922716, 100.26932691, ..., 100.1716221 ,
                              100.99366762, 104.09253633]),
              'x': array([10, 10, 10, ..., 90, 90, 90]),
              'y': array([ 50,  50,

In [259]:
pf.value.iloc[-1].sort_values(ascending=False)

si_period  si_period_band  si_multiplier
50         180             2.5              109.704776
           190             2.5              108.914127
           170             2.5              108.730288
70         190             2.5              108.300211
60         160             2.5              107.172152
                                               ...    
50         50              2.5               94.800654
20         80              2.0               94.678434
           140             2.0               94.596386
           160             2.0               93.970080
           150             2.0               93.961599
Name: 2024-12-01 23:59:00+00:00, Length: 540, dtype: float64

In [260]:
pf[(50, 180, 2.5)].plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'f41154a9-ba32-46b5-8c72-75e56312bd11',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100., 100., 100., ..., 100., 100., 100.])},
           

In [220]:
pf[(20, 150, 2.0)].plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '8e31de25-8810-47d4-8ea4-73f040a463b1',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100., 100., 100., ..., 100., 100., 100.])},
           

### Parameterised Pipeline

In [273]:
@vbt.parameterized(merge_func=lambda r: [x for inner in r for x in inner])
def f(x, y):
    return (x,y)

f(
    x=vbt.Param(np.arange(0, 2, 1)),
    y=vbt.Param(np.arange(0, 2, 1))
)

[np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1)]

In [ ]:
vbt.parameterized(pipeline)

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': '6dab8937-0aac-4803-a167-cc6169cf0b58',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100.        , 100.        , 100.        , ...,  96.99170444,
                           96.99566927,  96.99170444])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                   

In [371]:
@vbt.parameterized(merge_func='concat')
# @vbt.parameterized(merge_func='concat', execute_kwargs=dict(chunk_len='auto', engine='multiprocessing'))
def pipeline(close, period=10, period_band=60, multiplier=1.):
    SimpleIndicator =  make_indicator_with_func(simple_indicator_nb)
    indicator = SimpleIndicator.run(
        close,
        period=period,
        period_band=period_band,
        multiplier=multiplier,
    )
    long_entries = indicator.indicator < indicator.upper
    long_exits = indicator.indicator > indicator.lower
    short_entries = long_exits
    short_exits = long_entries
    pf = vbt.Portfolio.from_signals(
        close,
        long_entries=long_entries,
        long_exits=long_exits,
        short_entries=short_entries,
        short_exits=short_exits,
        fees=0.0,
        # freq="1d",
    )
    return pf.value.iloc[-1]

In [372]:
pf = pipeline(
    close,
    period=vbt.Param(np.arange(10, 101, 10)),
    period_band=vbt.Param(np.arange(50, 301, 10)),
    multiplier=vbt.Param(np.arange(1, 3, 0.1)),
    _execute_kwargs=dict(chunk_len='auto', engine='dask')
)

  1%|1         | 3/260 [00:02<02:58,  1.44it/s, chunk_tasks=60..79]

In [373]:
pf.vbt.heatmap(
    x_level='period',
    y_level='period_band',
    slider_level='multiplier'
)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'hovertemplate': 'period: %{x}<br>period_band: %{y}<br>value: %{z}<extra></extra>',
              'name': '1.0',
              'type': 'heatmap',
              'uid': 'c1567a8e-176c-4732-8c3e-00487729ef07',
              'visible': True,
              'x': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100]),
              'y': array([ 50,  60,  70,  80,  90, 100, 110, 120, 130, 140, 150, 160, 170, 180,
                          190, 200, 210, 220, 23